In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

FSA = []
for post in soup.find_all('td'):
    code = post.text
    if code == '':
        break
    else:
        last_chr = int(len(code))
        last_chr = last_chr - 1
        if code[last_chr] == '\n':
            code = code[:-1]
        FSA.append(code)
post_dict = {}
postcode = []
burrough = []
neighbourhood = []
while len(FSA) > 2:
    postcode.append(FSA[0])
    burrough.append(FSA[1])
    neighbourhood.append(FSA[2])
    del FSA[0:3]

post_dic = {'Postal Code':postcode, 'Burrough':burrough, 'Neighborhood':neighbourhood}

df_post = pd.DataFrame(post_dic)

df_post = df_post[df_post.Burrough != 'Not assigned']

df_post = df_post.groupby(['Postal Code', 'Burrough'])['Neighborhood'].apply(', '.join).reset_index()

df_post.loc[df_post.Neighborhood == 'Not assigned', 'Neighborhood'] = df_post.Burrough

#print(df_post)

df_post.shape

(103, 3)

In [9]:
df_geo = pd.read_csv('https://cocl.us/Geospatial_data')
df_new = pd.merge(df_post, df_geo, on='Postal Code')
df_new.head()

,Postal Code,Burrough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
address = 'Toronto, CN'

geolocater = Nominatim()
location = geolocater.geocode(address)
latitude = location.longitude
longitude = location.longitude
print('The geographical coordinate of Toronto, Cananda are {}, {}'.format(latitude, longitude))

C:\Users\cewhite\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geographical coordinate of Toronto, Cananda are -79.3870871832047, -79.3870871832047


In [11]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Burrough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto)
    
map_Toronto 
    

In [12]:
Toronto_west = df_new[df_new['Burrough'] == 'West Toronto'].reset_index(drop=True)
Toronto_east = df_new[df_new['Burrough'] == 'East Toronto'].reset_index(drop=True)
Toronto_down = df_new[df_new['Burrough'] == 'Downtown Toronto'].reset_index(drop=True)
Toronto_central = df_new[df_new['Burrough'] == 'Central Toronto'].reset_index(drop=True)
Toro = pd.concat([Toronto_west, Toronto_east,Toronto_down,Toronto_central])
Toro.reset_index(drop=True, inplace=True)
Toro

,Postal Code,Burrough,Neighborhood,Latitude,Longitude
0,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450
6,M4E,East Toronto,The Beaches,43.676357,-79.293031
7,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
8,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
9,M4M,East Toronto,Studio District,43.659526,-79.340923


In [13]:
map_Toronto_neigh = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat,lng, burough, neighborhood in zip(Toro['Latitude'], Toro['Longitude'], Toro['Burrough'], Toro['Neighborhood']):
    label = '{}, {}'.format(neighborhood, burough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat, lng],
                        radius=5,
                        popup=label,
                        color='blue',
                        fill=True,
                        fill_color='#3186cc',
                        fill_opacity=0.7,
                        parse_html=False).add_to(map_Toronto_neigh)
    
map_Toronto_neigh

In [14]:
CLIENT_ID = '3M5MD4WTFDBCTGKO2CFVH120NXIJ1R43IRBJTVU0BGGIMSUP'
CLIENT_SECRET = 'AJRZ10JWEYOTWU44WM3JC04XJKQN2Y3XB2T5DHS5JFUMOYGX'
VERSION = '20180605'

In [19]:
radius = 500
LIMIT = 100
#url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID,
#     CLIENT_SECRET,
#     VERSION,
#     neighborhood_latitude,
#     neighborhood_longitude,
#     radius,
#     LIMIT)

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat,
            lng, 
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                            'Neighborhood Latitude', 
                            'Neighborhood Longitude', 
                            'Venue', 
                            'Venue Latitude', 
                            'Venue Longitude', 
                            'Venue Category']
    return(nearby_venues)

In [26]:
#print(Toro['Neighborhood'][1])
Tor_ven = getNearbyVenues(names=Toro['Neighborhood'], latitudes=Toro['Latitude'], longitudes=Toro['Longitude'])


Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction South
Parkdale, Roncesvalles
Runnymede, Swansea
The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Business reply mail Processing Centre969 Eastern
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, Sou

In [27]:
print(Tor_ven.shape)
print(Tor_ven.head())

(1705, 7)
                   Neighborhood  Neighborhood Latitude  \
0  Dovercourt Village, Dufferin              43.669005   
1  Dovercourt Village, Dufferin              43.669005   
2  Dovercourt Village, Dufferin              43.669005   
3  Dovercourt Village, Dufferin              43.669005   
4  Dovercourt Village, Dufferin              43.669005   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.442259             The Greater Good Bar       43.669409   
1              -79.442259                         Parallel       43.669516   
2              -79.442259                          FreshCo       43.667918   
3              -79.442259          Happy Bakery & Pastries       43.667050   
4              -79.442259  Planet Fitness Toronto Galleria       43.667588   

   Venue Longitude             Venue Category  
0       -79.439267                        Bar  
1       -79.438728  Middle Eastern Restaurant  
2       -79.440754          

In [28]:
Tor_ven.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,53,53,53,53,53,53
"Brockton, Exhibition Place, Parkdale Village",21,21,21,21,21,21
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14
"Cabbagetown, St. James Town",48,48,48,48,48,48
Central Bay Street,82,82,82,82,82,82
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


In [29]:
print('There are {} unique categories.'.format(len(Tor_ven['Venue Category'].unique())))

There are 234 unique categories.


In [ ]:
toro_onehot = pd.get_dummies(Tor_ven[['Venue Category']])